In [1]:
import os
import json
import pandas as pd
from collections import defaultdict

directory = os.fsencode('data/wtt_matches')

In [2]:
tf = pd.read_csv('data/tournaments_wtt.tsv', sep='\t', parse_dates=['StartDateTime', 'EndDateTime'])
tf.sort_values(by=['StartDateTime'], inplace=True)

In [3]:
players = []
genders = defaultdict(lambda: None)

for row in tf.itertuples():
    evt = os.path.join(f'data/wtt_matches', str(row.EventId))
    if not os.path.isdir(evt):
        continue

    for filename in os.listdir(evt):
        if not filename.endswith(".json"):
            continue

        m = None
        with open(os.path.join(f'data/wtt_matches', str(row.EventId), filename)) as f:
            m = json.load(f)

        gender = m['documentCode'][3]

        isTeam = False
        for c in m['competitiors']:
            if isTeam:
                break

            for p in c['players']:
                p['gender'] = gender
                pid = int(p['playerId'])
                if pid > 100000000 or pid < 90000:
                    isTeam = True
                if isTeam:
                    break

                if not p['playerOrgCode']:
                    p['playerOrgCode'] = c['competitiorOrg']
                players.append(p)

        if isTeam and m['teamParentData']:
            for _m in m['teamParentData']['extended_info']['matches']:
                if not _m.get('match_result'):
                    continue

                mm = _m['match_result']
                for c in mm['competitiors']:
                    for p in c['players']:
                        p['gender'] = gender
                        pid = int(p['playerId'])
                        if pid > 100000000 or pid < 90000:
                            continue

                        if not p['playerOrgCode']:
                            p['playerOrgCode'] = c['competitiorOrg']
                        players.append(p)


In [4]:
pf = pd.DataFrame(players)
pf.drop(columns=['playerPosition'], inplace=True)
pf

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender
0,112074,FALCK Mattias,None,None,SWE,M
1,104379,KARLSSON Kristian,None,None,SWE,M
2,118994,JEON Jihee,Jihee,JEON,KOR,W
3,121706,Mariia TAILAKOVA,Mariia,TAILAKOVA,RUS,W
4,101648,CHEN Chien-An,None,None,TPE,M
...,...,...,...,...,...,...
58899,115920,BATRA Manika,None,None,IND,W
58900,107028,OVTCHAROV Dimitrij,None,None,GER,M
58901,134442,SHINOZUKA Hiroto,None,None,JPN,M
58902,122777,UDA Yukiya,None,None,JPN,M


In [5]:
pf[pf.playerName == 'Lubomir PISTEJ']

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender
191,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,X
432,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
497,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
549,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
919,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
1253,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,M
1606,107445,Lubomir PISTEJ,Lubomir,PISTEJ,SVK,X


In [6]:
pf[pf.playerId.isna()]
# only 2, manually fix

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender


In [7]:
name_conflicts = pf.groupby('playerId')['playerName'].apply(set)
conf = pd.DataFrame(name_conflicts)
conf['cnt'] = name_conflicts.apply(len)
conf[conf.cnt > 1]


,playerName,cnt
playerId,,
100032,"{Farah ABDELAZIZ, ABDEL-AZIZ Farah}",2
100439,"{SALEH Ahmed, Ahmed SALEH}",2
100486,"{Gaston ALTO, ALTO Gaston}",2
100621,"{Tiago Apolonia, Tiago APOLONIA, APOLONIA Tiago}",3
100696,"{ASSAR Omar, Omar Assar, Omar ASSAR}",3
...,...,...
200391,"{MENEZES ARAUJO Abimael, MENEZES Abimael}",2
200599,"{TOVAR Jesus, TOVAR GIRALDO Jesus Alejandro}",2
200649,"{VEGAS Jonathan, VEGAS VASQUEZ Jonathan Jeanca...",2


In [8]:
id_conflicts = pf.groupby('playerName')['playerId'].apply(set)
idconf = pd.DataFrame(id_conflicts)
idconf['cnt'] = id_conflicts.apply(len)
idconf[idconf.cnt > 1]
# These actually might be different people... 

,playerId,cnt
playerName,,
KUMAHARA Luca,"{206781, 113418}",2
LEE Daeun,"{135391, 132702}",2
LOPEZ Francisco,"{201761, 90051}",2
Nikhil KUMAR,"{119463, 116936}",2
Yang WANG,"{109995, 112735}",2


In [9]:
org_conflicts = pf.groupby('playerId')['playerOrgCode'].apply(set)
oconf = pd.DataFrame(org_conflicts)
oconf['cnt'] = org_conflicts.apply(len)
oconf[oconf.cnt > 1]


,playerOrgCode,cnt
playerId,,
100868,"{SVK/CZE, LUX/SVK, SVK/USA, SVK, SVK/FRA, CZE/...",7
102380,"{ENG, GBR, ENG/GER}",3
103163,"{USA/SUI, USA/ESP, USA}",3
103425,"{CMR, CMR/LUX}",2
103903,"{ROU, ROU/ESP}",2
...,...,...
146048,"{PER/ECU, PER}",2
200322,"{KAZ, LUX/KAZ}",2
200904,"{FIN, SWE/FIN, EGY/FIN}",3


In [10]:
cleaned = []
for id, rows in pf.groupby('playerId'):
    clrow = {
        'id': id,
        'org': None,
        'name': '',
        'gender': 'X',
    }
    for row in rows.itertuples():
        if row.gender != 'X':
            clrow['gender'] = row.gender
        if '^' in clrow['name'] or (clrow['name'] != row.playerName and min(sum(1 for c in clrow['name'] if c.isupper()), 4) < min(sum(1 for c in row.playerName if c.isupper()), 4)):
            clrow['name'] = row.playerName
        if not clrow['org']:
            clrow['org'] = row.playerOrgCode
        elif row.playerOrgCode:
            # take shorter country code, don't want doubles codes
            if len(row.playerOrgCode) < len(clrow['org']):
                clrow['org'] = row.playerOrgCode

    cleaned.append(clrow)
cf = pd.DataFrame(cleaned)

In [11]:
pf[pf.playerId.isin(cf[cf.org.isna()].id)]

,playerId,playerName,playerGivenName,playerFamilyName,playerOrgCode,gender


In [12]:
cf.to_csv('data/wtt_cleaned/players.tsv', index=False, sep='\t')